### **Exporting VIRCAM confidence map metadata**

This notebook scans the VIDEO VIRCAM confidence maps (*_st_conf.fit) and produces an ECSV file that can later be ingested into the LSST Butler.

Steps performed:

1. Find input FITS files
   
   - Looks under data/*/*_st_conf.fit.

   - For each file, a relative path of the form:
   ../../dmu0/dmu0_VISTA/dmu0_VIDEO_CSDF/... is written to match the Butler ingest location.

2. Read main FITS header

   - Extracts the filter band from ESO INS FILT1 NAME.

   - Normalizes physical filter names (e.g. K → VIRCAM-Ks, otherwise VIRCAM-<band>).

3. Read detector header

   - Uses ESO DET EXP NO from the first detector extension to obtain the unique exposure ID.

4. Write metadata table

5. Save results: test_export_confidence.ecsv

In [29]:
import glob
import os
from astropy.io import fits
from astropy.table import Table

rows = []

# Match all st_conf files relative to current folder
for st_file in sorted(glob.glob("data/*/*_st_conf.fit")):
    try:
        with fits.open(st_file) as hdul:
            hdr0 = hdul[0].header  # main header
            band = hdr0.get("ESO INS FILT1 NAME", "").strip()
            if band == "K":
                physical_filter = "VIRCAM-Ks"
            else:
                physical_filter = f"VIRCAM-{band}"
            
            # Exposure ID from detector header
            exp_id = hdul[1].header["ESO DET EXP NO"]

            # Build the wanted path
            rel_path = os.path.join("../../../dmu0/dmu0_VISTA/dmu0_VIDEO_CSDF", st_file)

            for det in range(1, 17):
                rows.append([
                    rel_path,
                    "VIRCAM",
                    band,
                    physical_filter,
                    exp_id,
                    det
                ])
    except Exception as e:
        print(f"⚠️ Problem reading {st_file}: {e}")

# Build Astropy table
t = Table(
    rows=rows,
    names=["filename", "instrument", "band", "physical_filter", "exposure", "detector"]
)

# Save to ECSV
t.write("example_export_confidence.ecsv", format="ascii.ecsv", overwrite=True)
print("✅ Wrote example_export_confidence.ecsv with", len(rows), "rows")

✅ Wrote example_export_confidence.ecsv with 320 rows
